# Benchmarking comparison of VAE Model with CellAssign 

The dataset used for this comparison is the Follicular Lymphoma dataset, which is one of the datasets used to showcase the step-by-step implementation of CellAssign in the Cell Assign Tutorial Page of the "scvi-tools" documentation: https://docs.scvi-tools.org/en/stable/tutorials/notebooks/cellassign_tutorial.html.

The CellAssign tutorial demonstration uses the Follicular Lymphoma dataset that was used in the original publication of CellAssign: https://www.nature.com/articles/s41592-019-0529-1. The authors of the Tutorial converted the original data into the h5ad format, which can be found here: https://drive.google.com/uc?id=10l6m2KKKioCZnQlRHomheappHh-jTFmx. We used this h5ad formatted data to benchmark the performance of our VAE model. 

# Load packages

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!pip install scanpy
!pip install matplotlib==3.1.3
!pip install -U tensorflow
!pip install celltypist

import numpy as np 
import pandas as pd 
import scanpy as sc
import seaborn as sns
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import math
import scipy
import tensorflow as tf
from tensorflow.keras import layers, losses
from tensorflow.keras.models import Model
import celltypist
from celltypist import models

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached matplotlib-3.5.3-cp37-cp37m-manylinux_2_5_x86_64.manylinux1_x86_64.whl (11.2 MB)
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.1.3
    Uninstalling matplotlib-3.1.3:
      Successfully uninstalled matplotlib-3.1.3


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached matplotlib-3.1.3-cp37-cp37m-manylinux1_x86_64.whl (13.1 MB)
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.5.3
    Uninstalling matplotlib-3.5.3:
      Successfully uninstalled matplotlib-3.5.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
scanpy 1.9.1 requires matplotlib>=3.4, but you have matplotlib 3.1.3 which is incompatible.


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached matplotlib-3.5.3-cp37-cp37m-manylinux_2_5_x86_64.manylinux1_x86_64.whl (11.2 MB)
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.1.3
    Uninstalling matplotlib-3.1.3:
      Successfully uninstalled matplotlib-3.1.3


In [ ]:
!pip uninstall matplotlib
!pip install matplotlib==3.1.3

Found existing installation: matplotlib 3.5.3
Uninstalling matplotlib-3.5.3:
  Would remove:
    /usr/local/lib/python3.7/dist-packages/matplotlib-3.5.3-py3.7-nspkg.pth
    /usr/local/lib/python3.7/dist-packages/matplotlib-3.5.3.dist-info/*
    /usr/local/lib/python3.7/dist-packages/matplotlib/*
    /usr/local/lib/python3.7/dist-packages/mpl_toolkits/axes_grid/*
    /usr/local/lib/python3.7/dist-packages/mpl_toolkits/axes_grid1/*
    /usr/local/lib/python3.7/dist-packages/mpl_toolkits/axisartist/*
    /usr/local/lib/python3.7/dist-packages/mpl_toolkits/mplot3d/*
    /usr/local/lib/python3.7/dist-packages/mpl_toolkits/tests/*
    /usr/local/lib/python3.7/dist-packages/pylab.py
Proceed (y/n)? y
  Successfully uninstalled matplotlib-3.5.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached matplotlib-3.1.3-cp37-cp37m-manylinux1_x86_64.whl (13.1 MB)
ERROR: pip's dependency resolver does not currently take into account all the pa

# Define VAE class

In [ ]:
# Create weight constraint so that marker gene weights are non-negative
class BiasedNonNegative(tf.keras.constraints.Constraint):

  def __init__(self, weight_mask):
    self.marker_weight_mask = weight_mask
    self.NonMarker_weight_mask = 1 - weight_mask

# Multiply the weight matrix by weight mask
  def __call__(self, w):
    marker_sparse_w = w * self.marker_weight_mask                                                            # make marker weights sparse
    NonNeg_marker_sparse_w = marker_sparse_w * tf.cast(tf.math.greater_equal(marker_sparse_w, 0.), w.dtype)  # sparse * non-negativity constraint 
    NonMarker_sparse_w = w * self.NonMarker_weight_mask                                                      # make non-marker weights sparse
    return NonNeg_marker_sparse_w + NonMarker_sparse_w                                                       # add non-marker weights regardless of non-negativity

In [ ]:
class SparseNonNegative(tf.keras.constraints.Constraint):

  def __init__(self, weight_mask):
    self.weight_mask = weight_mask

# Multiply the weight matrix by weight mask

  def __call__(self, w):
    w = w * self.weight_mask * tf.cast(tf.math.greater_equal(w, 0.), w.dtype)    # sparse * non-negative 
    return w                   

In [ ]:
class BiasedVAE(Model):
    def __init__(self, num_cell_types, num_genes, reg_penalties, weight_mask):
        super(BiasedVAE, self).__init__()  
        self.reg_penalties = reg_penalties
        self.encoder_weight_mask = tf.keras.backend.constant(weight_mask) 
        self.decoder_weight_mask = tf.keras.backend.constant(weight_mask.T)
        self.mean_encoder = tf.keras.Sequential([
         layers.Dense(num_cell_types, activation='linear', 
                      kernel_regularizer=self.encoder_marker_gene_regularizer, 
                      kernel_constraint=BiasedNonNegative(self.encoder_weight_mask)),
        ])
        self.var_encoder = tf.keras.Sequential([
         layers.Dense(num_cell_types, activation='linear', 
                      kernel_regularizer=self.encoder_marker_gene_regularizer, 
                      kernel_constraint=BiasedNonNegative(self.encoder_weight_mask)),
        ])
        self.decoder = tf.keras.Sequential([
          layers.Dense(num_genes, activation='linear', 
                      kernel_regularizer=self.decoder_marker_gene_regularizer, 
                      kernel_constraint=BiasedNonNegative(self.decoder_weight_mask))
        ])

    def reparameterization(self, mean, var):
        epsilon = tf.random.normal(shape = [mean.shape[1]])             
        z = mean + (var*epsilon)                       
        return z
        
    def call(self, x):
        mean = self.mean_encoder(x)
        log_var = self.var_encoder(x)
        z = self.reparameterization(mean, tf.exp(0.5 * log_var))
        p = tf.keras.activations.softmax(z)
        decoded = self.decoder(p)
        return decoded

    def embed(self, x):
        mean = self.mean_encoder(x)
        log_var = self.var_encoder(x)
        var = tf.exp(0.5 * log_var)
        return tf.keras.activations.softmax(mean), mean, var

    def encoder_marker_gene_regularizer(self, w):
        return tf.reduce_sum(self.reg_penalties * tf.square(w))

    def decoder_marker_gene_regularizer(self, w):
        return tf.reduce_sum(self.reg_penalties.T * tf.square(w))

In [ ]:
class BiasedAE(Model):
    def __init__(self, num_cell_types, num_genes, reg_penalties, weight_mask):
        super(BiasedAE, self).__init__()  
        self.reg_penalties = reg_penalties
        self.encoder_weight_mask = weight_mask 
        self.decoder_weight_mask = weight_mask.T
        self.encoder = tf.keras.Sequential([
         layers.Dense(num_cell_types, activation='softmax', 
                      kernel_regularizer=self.encoder_marker_gene_regularizer, 
                      kernel_constraint=BiasedNonNegative(self.encoder_weight_mask)),
        ])
        self.decoder = tf.keras.Sequential([
          layers.Dense(num_genes, activation='linear', 
                      kernel_regularizer=self.decoder_marker_gene_regularizer, 
                      kernel_constraint=BiasedNonNegative(self.decoder_weight_mask))
        ])
        
    def call(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

    def embed(self, x):
      encoded = self.encoder(x)
      return encoded

    def encoder_marker_gene_regularizer(self, w):
      return tf.reduce_sum(self.reg_penalties * tf.square(w))

    def decoder_marker_gene_regularizer(self, w):
      return tf.reduce_sum(self.reg_penalties.T * tf.square(w))

In [ ]:
class SparseVAE(Model):
    def __init__(self, num_cell_types, num_genes, weight_mask):
        super(SparseVAE, self).__init__()  
        self.encoder_weight_mask = tf.keras.backend.constant(weight_mask) 
        self.decoder_weight_mask = tf.keras.backend.constant(weight_mask.T)
        self.mean_encoder = tf.keras.Sequential([
         layers.Dense(num_cell_types, activation='linear', 
                      kernel_constraint=SparseNonNegative(self.encoder_weight_mask)),
        ])
        self.var_encoder = tf.keras.Sequential([
         layers.Dense(num_cell_types, activation='linear', 
                      kernel_constraint=SparseNonNegative(self.encoder_weight_mask)),
        ])
        self.decoder = tf.keras.Sequential([
          layers.Dense(num_genes, activation='linear', 
                      kernel_constraint=SparseNonNegative(self.decoder_weight_mask))
        ])

    def reparameterization(self, mean, var):
        epsilon = tf.random.normal(shape = [mean.shape[1]])             
        z = mean + (var*epsilon)                       
        return z
        
    def call(self, x):
        mean = self.mean_encoder(x)
        log_var = self.var_encoder(x)
        z = self.reparameterization(mean, tf.exp(0.5 * log_var))
        p = tf.keras.activations.softmax(z)
        decoded = self.decoder(p)
        return decoded

    def embed(self, x):
        mean = self.mean_encoder(x)
        log_var = self.var_encoder(x)
        var = tf.exp(0.5 * log_var)
        return tf.keras.activations.softmax(mean), mean, var

In [ ]:
class SparseAE(Model):
    def __init__(self, num_cell_types, num_genes, weight_mask):
        super(SparseAE, self).__init__()  
        self.encoder_weight_mask = weight_mask 
        self.decoder_weight_mask = weight_mask.T
        self.encoder = tf.keras.Sequential([
          layers.Dense(num_cell_types, 
                       activation='softmax', 
                       kernel_constraint=SparseNonNegative(self.encoder_weight_mask))
          ])
        self.decoder = tf.keras.Sequential([
          layers.Dense(num_genes, 
                       activation='linear', 
                       kernel_constraint = SparseNonNegative(self.decoder_weight_mask))
        ])
        
    def call(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

    def embed(self, x):
      encoded = self.encoder(x)
      return encoded

# Define t-test computation function

In [ ]:
def Compute_tTest(mean1,var1,mean2,var2,n):
  tScore = (mean1-mean2) / math.sqrt((var1+var2)/n)
  DegFreedom = ((((var1**2)+(var2**2))/n)**2) / ((((var1**2)+(var2**2))/(n**2))/(n-1))
  return scipy.stats.t.sf(abs(tScore), df=DegFreedom)

# Cell Type Assignment Function

In [ ]:
from numpy.ma.core import true_divide
def CellTypeAssignment(dataset, Model = "Biased VAE", LearningRate = 1e-4, pValue = 2, MarkerBias = 1, NumEpochs = 50, NumTopGenes = 10):

    # Load Panglao DB marker gene list 
    print('Cell types with marker genes available in Panglao DB: \n')
    MarkerGeneList = pd.read_csv("/content/drive/MyDrive/ML for genomics project group /Data/PanglaoDB_Markers.tsv",sep='\t')

    # Print expected cell type options 
    print(*list(pd.unique(MarkerGeneList['cell type'])), sep = "\n")
    print('\n')

    CellMarkers = pd.DataFrame(columns = MarkerGeneList.columns)
    PanglaoDB_list = CellMarkers['official gene symbol']
    AddCellType = "Y"
    while AddCellType == "Y":
        AddCellType = input("Add expected cell type: Y or N? ")
        while AddCellType not in ["Y","N"]:
            AddCellType = input("Add expected cell type: Y or N? ")
        if AddCellType == "Y":
            CellTypeOption = 'K'
            while CellTypeOption not in ["C", "P"]:
                CellTypeOption = input("To add custom expected cell type, type 'C'. To add expected cell type from Panglao DB, type 'P': ")
            if CellTypeOption == 'P':
                CellTypeInputRepeat = True
                while CellTypeInputRepeat == True:
                    TempCellType = input("Expected cell type: ")
                    if TempCellType not in list(pd.unique(CellMarkers['cell type'])):
                        CellTypeInputRepeat = False
                    else:
                        print("Expected cell type already inputted!")
                if TempCellType in list(pd.unique(MarkerGeneList['cell type'])): 
                    CellMarkers = CellMarkers.append(MarkerGeneList.iloc[np.asarray(np.where(MarkerGeneList['cell type'] == TempCellType))[0,:],:])
                else: 
                    print("The cell type inputted is not available in the Panglao DB database!")
                PanglaoDB_list = list(set(CellMarkers['official gene symbol']) & set(dataset.var_names))
                CellMarkers_list = []
                for gene in PanglaoDB_list:
                  if gene in list(CellMarkers['official gene symbol']):
                    CellMarkers_list.append({
                        'official gene symbol': gene,
                        'cell type': CellMarkers.loc[CellMarkers['official gene symbol'] == gene, 'cell type'].iloc[0]
                    })
                CellMarkers = pd.DataFrame(CellMarkers_list)
            else:
                CellTypeInputRepeat = True
                while CellTypeInputRepeat == True:
                    TempCellType = input("Expected cell type: ")
                    if TempCellType not in list(pd.unique(CellMarkers['cell type'])):
                        CellTypeInputRepeat = False
                    else:
                        print("Expected cell type already inputted!")
                CellTypeMarkerGenes = []
                KeepAdding = 'Y'
                while KeepAdding == 'Y':
                    AddMarker = input("Add marker gene: ")
                    if ((AddMarker in dataset.var_names) & (AddMarker not in CellMarkers['official gene symbol'])):
                        CellTypeMarkerGenes.append(AddMarker)
                    else:
                        print("Marker gene not available!")
                    KeepAdding = 'K'
                    while KeepAdding not in ['Y','N']:
                        KeepAdding = input("Add more markers: Y or N? ")
                CellType_df = pd.DataFrame({"official gene symbol":CellTypeMarkerGenes, 
                                            "cell type":[TempCellType]*len(CellTypeMarkerGenes)})
                PanglaoDB_list = list(set(PanglaoDB_list) & set(dataset.var_names))
                PanglaoDB_list = PanglaoDB_list + list(CellType_df['official gene symbol'])
                CellMarkers_list = []
                for gene in PanglaoDB_list:
                  if gene in list(CellMarkers['official gene symbol']):
                    CellMarkers_list.append({
                        'official gene symbol': gene,
                        'cell type': CellMarkers.loc[CellMarkers['official gene symbol'] == gene, 'cell type'].iloc[0]
                    })
                CellMarkers = pd.DataFrame(CellMarkers_list)
                CellMarkers = pd.concat([CellMarkers, CellType_df])   
    
    PanglaoDB_list = list(CellMarkers['official gene symbol'])

    print("\n")
    print(CellMarkers)
    print("\n")
    RenameCellTypes = 'Y'
    while RenameCellTypes == 'Y':
        print("Expected cell types: ")
        print(*list(pd.unique(CellMarkers['cell type'])), sep = ', ')
        print("\n")

        RenameCellTypes = 'K'
        while RenameCellTypes not in ['Y','N']:
            RenameCellTypes = input("Would you like to rename any expected cell types: Y or N? ")
            if RenameCellTypes == 'Y':
                ReplacementAvailable = 'N'
                while ReplacementAvailable == 'N':
                    ReplacedCellType = input("Expected cell type to replace: ")
                    if ReplacedCellType in list(pd.unique(CellMarkers['cell type'])):
                        ReplacementAvailable = 'Y'
                    else:
                        print('Expected cell type not available in current list!')
                Replacement = input("Replacement cell type: ")
                CellMarkers['cell type'] = CellMarkers['cell type'].replace(ReplacedCellType, Replacement)

    # Create a variable list for input genes / features 
    PanglaoDB_Input = []

    for i in range(dataset.shape[1]):
      if dataset.var_names[i] in PanglaoDB_list:
        PanglaoDB_Input.append(True)
      else: 
        PanglaoDB_Input.append(False)

    dataset.var['input_features'] = PanglaoDB_Input

    # Create anndata input for the model by filtering dataset
    dataset_input = dataset[:,dataset.var['input_features']]

    # Creating weight mask for marker genes
    marker_genes = dataset_input.var_names
    cell_types = pd.unique(CellMarkers['cell type'])

    marker_gene_cell_type_weight_mask = np.zeros((len(marker_genes), len(cell_types)))
    CellTypeMaskLabel = 0
    for i, gene in enumerate(marker_genes):
      for j, cell_type in enumerate(cell_types):
        is_marker_gene_for_cell_type = \
          CellMarkers[(CellMarkers['official gene symbol'] == gene) & 
                          (CellMarkers['cell type'] == cell_type)].shape[0] > 0
        if is_marker_gene_for_cell_type:   # Mask = 1
          marker_gene_cell_type_weight_mask[i, j] = 1
        else:                              # Mask = 0
          marker_gene_cell_type_weight_mask[i, j] = 0

    # Creating marker gene biased regularization mask 
    marker_gene_cell_type_reg_penalties = (1 - marker_gene_cell_type_weight_mask)*MarkerBias

    # Train the model 
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    NumCellTypes = len(CellMarkers['cell type'].unique())
    num_genes = dataset_input.shape[1]
    print(Model)
    if Model == "Biased VAE": 
        autoencoder = BiasedVAE(NumCellTypes, num_genes, 
                                marker_gene_cell_type_reg_penalties,
                                marker_gene_cell_type_weight_mask)
    elif Model == "Biased AE":
        autoencoder = BiasedAE(NumCellTypes, num_genes, 
                              marker_gene_cell_type_reg_penalties,
                              marker_gene_cell_type_weight_mask)
    elif Model == "Sparse VAE":
        autoencoder = SparseVAE(NumCellTypes, num_genes, 
                              marker_gene_cell_type_weight_mask)
    elif Model == "Sparse AE":
        autoencoder = SparseAE(NumCellTypes, num_genes,
                              marker_gene_cell_type_weight_mask)
    else:
        print('ERROR: select either Biased VAE, Biased AE, Sparse VAE or Sparse AE as your model.')
        return
    autoencoder.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=LearningRate), 
                        loss=losses.MeanSquaredError())
    
    print('\n')
    print('Training the autoencoder model...')
    print('\n')

    history = autoencoder.fit(dataset_input.X, dataset_input.X,
                              epochs=NumEpochs,
                              shuffle=True,
                              validation_data=(dataset_input.X, dataset_input.X))
    
    plt.figure(figsize=(8, 5))
    plt.plot(history.history['loss'])
    plt.title('Training loss over epochs')
    plt.ylabel('Mean squared error')
    plt.xlabel('Epochs')
    
    # Embed the model 
    if Model in ["Biased VAE","Sparse VAE"]: 
        latent_embedding, mean, var = autoencoder.embed(dataset_input.X)
    elif Model in ["Biased AE","Sparse AE"]:
        latent_embedding = autoencoder.embed(dataset_input.X)

    # assign cell type labels based on maximum probability in the latent layer output 
    cell_type_labels = []
    for item in latent_embedding:
        cell_type_labels.append(cell_types[np.where(item == max(item))[0][0]])

    # compute pValue of each assignment and assign 'unknown' labels according p-value threshold
    if Model in ["Biased VAE","Sparse VAE"]: 
        print('\n')
        print('Calculating p-value of cell type assignments...')
        print('\n')

        NumCells = dataset_input.shape[0]
        Cell_Assignment_PValues = []
        pVal_Threshold = pValue

        for cell in range(NumCells):
            MaxIndex = tf.math.argmax(latent_embedding[cell,:]).numpy()
            pVal = []
            for LatentNode in range(NumCellTypes):
              if LatentNode != MaxIndex:
                pVal.append(Compute_tTest(mean[cell,MaxIndex],var[cell,MaxIndex],
                                          mean[cell,LatentNode],var[cell,LatentNode],
                                          num_genes))
            Cell_Assignment_PValues.append(pVal[np.argmax(pVal)])
            if pVal[np.argmax(pVal)] > pVal_Threshold:
              print('Not Confident: ' + cell_type_labels[cell] + ' = ' + str(pVal[np.argmax(pVal)]))
              cell_type_labels[cell] = 'Unknown'

        Cell_Assignment_PValues = np.array(Cell_Assignment_PValues).ravel()
        dataset.obs['pVal'] = Cell_Assignment_PValues
        dataset.obs['Predicted Cell Type'] = cell_type_labels
    else: 
        dataset.obs['Predicted Cell Type'] = cell_type_labels

    # Plot UMAP results 
    print('\n')
    print('Creating UMAP of cell type predictions...')
    print('\n')
    sc.tl.pca(dataset, use_highly_variable=False)
    # sc.tl.pca(dataset, svd_solver='arpack', use_highly_variable=False, n_comps = NumCellTypes)
    # dataset.obsm['X_pca'] = mean.numpy()
    sc.pp.neighbors(dataset)
    # sc.pp.neighbors(dataset, n_neighbors=10, n_pcs=NumCellTypes)
    sc.tl.umap(dataset)
    sc.pl.umap(dataset, color=['Predicted Cell Type'])

    # Plot gene weight activation 
    print('\n')
    print('Heatmap of Gene Activation...')
    print('\n')
    sns.heatmap(autoencoder.layers[0].get_weights()[0],
            xticklabels = cell_types,
            yticklabels = marker_genes)
    plt.show()

    # Print top 10 most activated genes for each cell type assignment 
    print('\n')
    print('Most Activated Genes for Each Cell...')
    print('\n')
    i = 0 
    for cell in cell_types:
        print(cell_types[i] + ': \n')
        j = 0
        for m in marker_genes[np.argsort(autoencoder.layers[0].get_weights()[0][:,i])[-NumTopGenes:]]:
            if marker_gene_cell_type_weight_mask[:,i][np.argsort(autoencoder.layers[0].get_weights()[0][:,i])[-NumTopGenes:]][j] == 0:
                print(m + ' (UNIDENTIFIED MARKER)')
            else:
                print(m)
            j += 1
        print('\n')
        i += 1 

    return dataset, autoencoder, CellMarkers

# Follicular Lymphoma 

In [ ]:
SceFollicula_adata = sc.read_h5ad("/content/drive/MyDrive/ML for genomics project group /Data/CellAssign Data /Source Data/sce_follicular_annotated_final.h5ad")
SceFollicula_adata

AnnData object with n_obs × n_vars = 9156 × 33694
    obs: 'Sample', 'dataset', 'patient', 'timepoint', 'progression_status', 'patient_progression', 'sample_barcode', 'is_cell_control', 'total_features_by_counts', 'log10_total_features_by_counts', 'total_counts', 'log10_total_counts', 'pct_counts_in_top_50_features', 'pct_counts_in_top_100_features', 'pct_counts_in_top_200_features', 'pct_counts_in_top_500_features', 'total_features_by_counts_endogenous', 'log10_total_features_by_counts_endogenous', 'total_counts_endogenous', 'log10_total_counts_endogenous', 'pct_counts_endogenous', 'pct_counts_in_top_50_features_endogenous', 'pct_counts_in_top_100_features_endogenous', 'pct_counts_in_top_200_features_endogenous', 'pct_counts_in_top_500_features_endogenous', 'total_features_by_counts_feature_control', 'log10_total_features_by_counts_feature_control', 'total_counts_feature_control', 'log10_total_counts_feature_control', 'pct_counts_feature_control', 'pct_counts_in_top_50_features_featur

In [ ]:
SceFollicula_adata.obs

,Sample,dataset,patient,timepoint,progression_status,patient_progression,sample_barcode,is_cell_control,total_features_by_counts,log10_total_features_by_counts,...,all_seurat_cluster,all_seurat_0.8_cluster,all_seurat_1.2_cluster,all_sc3_cluster,all_SC3_cluster,all_cluster,all_subset_seurat_cluster,all_subset_seurat_0.8_cluster,all_subset_seurat_1.2_cluster,all_subset_cluster
Barcode,,,,,,,,,,,,,,,,,,,,,
AAACCTGAGCCACGTC-1,/datadrive/data/follicular/FL1018T1/filtered_g...,FL1018T1,FL1018,T1,primary,transformed,FL1018T1_AAACCTGAGCCACGTC-1,0,1300,3.114277,...,4,5,4,11,11,5,9,7,9,7
AAACCTGAGGGCTCTC-1,/datadrive/data/follicular/FL1018T1/filtered_g...,FL1018T1,FL1018,T1,primary,transformed,FL1018T1_AAACCTGAGGGCTCTC-1,0,846,2.927883,...,4,5,4,11,11,5,13,11,13,11
AAACCTGCACGAAGCA-1,/datadrive/data/follicular/FL1018T1/filtered_g...,FL1018T1,FL1018,T1,primary,transformed,FL1018T1_AAACCTGCACGAAGCA-1,0,1184,3.073718,...,4,5,4,19,19,5,0,1,0,1
AAACCTGCAGTTAACC-1,/datadrive/data/follicular/FL1018T1/filtered_g...,FL1018T1,FL1018,T1,primary,transformed,FL1018T1_AAACCTGCAGTTAACC-1,0,2019,3.305351,...,4,5,4,11,11,5,3,4,3,4
AAACCTGCATTCCTGC-1,/datadrive/data/follicular/FL1018T1/filtered_g...,FL1018T1,FL1018,T1,primary,transformed,FL1018T1_AAACCTGCATTCCTGC-1,0,1407,3.148603,...,8,7,8,27,27,7,6,2,6,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTTGTCACAAGACGTG-1,/datadrive/data/follicular/FL2001T2/filtered_g...,FL2001T2,FL2001,T2,progressed,progressed,FL2001T2_TTTGTCACAAGACGTG-1,0,2162,3.335057,...,7,6,7,11,11,6,5,5,5,5
TTTGTCACAAGGTTTC-1,/datadrive/data/follicular/FL2001T2/filtered_g...,FL2001T2,FL2001,T2,progressed,progressed,FL2001T2_TTTGTCACAAGGTTTC-1,0,737,2.868056,...,5,4,5,38,38,4,11,9,11,9
TTTGTCAGTCGAGATG-1,/datadrive/data/follicular/FL2001T2/filtered_g...,FL2001T2,FL2001,T2,progressed,progressed,FL2001T2_TTTGTCAGTCGAGATG-1,0,1376,3.138934,...,7,6,7,36,36,6,11,9,11,9


In [ ]:
# Subsetting the cell barcodes 
SceFollicula_adata.obs = SceFollicula_adata.obs[['Sample']]
SceFollicula_adata.obs.drop(['Sample'], axis=1, inplace=True)
SceFollicula_adata.obs

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


""
Barcode
AAACCTGAGCCACGTC-1
AAACCTGAGGGCTCTC-1
AAACCTGCACGAAGCA-1
AAACCTGCAGTTAACC-1
AAACCTGCATTCCTGC-1
...
TTTGTCACAAGACGTG-1
TTTGTCACAAGGTTTC-1
TTTGTCAGTCGAGATG-1


In [ ]:
# Load ground truth cell type labels 
SceFollicula_GroundTruth = pd.read_csv("/content/drive/MyDrive/ML for genomics project group /Data/CellAssign Data /GroundTruth_FollicularLymphoma.csv")
SceFollicula_adata.obs['Cell Type'] = SceFollicula_GroundTruth['x'].values
SceFollicula_adata.obs

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,Cell Type
Barcode,
AAACCTGAGCCACGTC-1,B cells (malignant)
AAACCTGAGGGCTCTC-1,B cells (malignant)
AAACCTGCACGAAGCA-1,B cells (malignant)
AAACCTGCAGTTAACC-1,B cells (malignant)
AAACCTGCATTCCTGC-1,CD4 T cells
...,...
TTTGTCACAAGACGTG-1,B cells (malignant)
TTTGTCACAAGGTTTC-1,B cells (malignant)
TTTGTCAGTCGAGATG-1,B cells (malignant)


In [ ]:
# Subsetting the gene name IDs 
SceFollicula_adata.var = SceFollicula_adata.var[['ID']]
SceFollicula_adata.var.set_index('ID', inplace=True)
SceFollicula_adata.var

""
ID
ENSG00000243485
ENSG00000237613
ENSG00000186092
ENSG00000238009
ENSG00000239945
...
ENSG00000277856
ENSG00000275063
ENSG00000271254


In [ ]:
# Load gene list 
FollicularLymphomaGenes = pd.read_csv("/content/drive/MyDrive/ML for genomics project group /Data/CellAssign Data /Gene_Name.txt")
FollicularLymphomaGenelist = []
for item in FollicularLymphomaGenes.values.tolist():
  FollicularLymphomaGenelist.append(item[0])
SceFollicula_adata.var_names = FollicularLymphomaGenelist

In [ ]:
SceFollicula_adata.var

,mean,std
ENSG00000243485,0.000000,1.000000
FAM138A,0.000000,1.000000
OR4F5,0.000000,1.000000
LOC100996442,0.002184,0.048972
ENSG00000239945,0.000328,0.018099
...,...,...
ENSG00000277856,0.000328,0.018099
LOC102723407,0.000109,0.010451
LOC102724250,0.007427,0.087126
ENSG00000277475,0.000000,1.000000


In [ ]:
sc.pp.scale(SceFollicula_adata)
SceFollicula_adata

AnnData object with n_obs × n_vars = 9156 × 33694
    obs: 'Cell Type'
    var: 'mean', 'std'
    uns: 'log.exprs.offset'
    obsm: 'X_pca', 'X_tsne', 'X_umap'
    layers: 'logcounts'

## Expected Cell Types 
The expected cell types for this dataset are: 
1. B cells (malignant) 
2. CD4 T cells
3. Cytotoxic T cells
4. Tfh
5. B cells
5. other 


In [ ]:
SceFollicula_adata.obs['Cell Type'].unique()

array(['B cells (malignant)', 'CD4 T cells', 'Cytotoxic T cells', 'Tfh',
       'B cells', 'other'], dtype=object)

In [ ]:
SceFollicula_adata, SceFollicula_VAE, SceFolliculaMarkers = CellTypeAssignment(SceFollicula_adata, 
                                                                   Model = "Sparse AE",
                                                                   LearningRate = 1e-3,
                                                                   MarkerBias = 1, 
                                                                   pValue = 2, 
                                                                   NumEpochs = 100)

Cell types with marker genes available in Panglao DB: 

Acinar cells
Adipocyte progenitor cells
Adipocytes
Adrenergic neurons
Airway epithelial cells
Airway goblet cells
Airway smooth muscle cells
Alpha cells
Alveolar macrophages
Anterior pituitary gland cells
Astrocytes
B cells
B cells memory
B cells naive
Basal cells
Basophils
Bergmann glia
Beta cells
Cajal-Retzius cells
Cardiac stem and precursor cells
Cardiomyocytes
Cholangiocytes
Cholinergic neurons
Chondrocytes
Choroid plexus cells
Chromaffin cells
Ciliated cells
Clara cells
Crypt cells
Decidual cells
Delta cells
Dendritic cells
Distal tubule cells
Dopaminergic neurons
Ductal cells
Embryonic stem cells
Endothelial cells
Endothelial cells (aorta)
Endothelial cells (blood brain barrier)
Enteric glia cells
Enteric neurons
Enterochromaffin cells
Enterocytes
Enteroendocrine cells
Eosinophils
Ependymal cells
Epiblast cells
Epithelial cells
Epsilon cells
Erythroblasts
Erythroid-like and erythroid precursor cells
Fibroblasts
Follicular c

KeyboardInterrupt: ignored